# Example of Weak Label Learning with warm up

In [1]:
import torch
import math
from datasets import openml_datasets,torch_datasets
from utils import weakener, losses, trainig_testing
from models import general_model

In [2]:
DS = openml_datasets.OpenML_Dataset('iris')
train_x, train_y, test_x, test_y = DS.get_data()

In [3]:
print(DS.num_features)
print(DS.num_classes)

4
tensor(3, dtype=torch.int32)


In [4]:
Weak = weakener.Weakener(DS.num_classes)

In [5]:
print(dir(Weak))

['M', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'c', 'generate_M', 'generate_weak', 'generate_wl_priors', 'label_matrix', 'pll_weights', 'virtual_labels', 'virtual_matrix', 'w', 'z']


In [6]:
print(DS.__dict__)

{'dataset': 'iris', 'tr_size': 0.7, 'weak_labels': None, 'batch_size': 64, 'shuffle': True, 'splitting_seed': 47, 'num_classes': tensor(3, dtype=torch.int32), 'train_dataset': <torch.utils.data.dataset.TensorDataset object at 0x0000021B8B179D60>, 'test_dataset': <torch.utils.data.dataset.TensorDataset object at 0x0000021B8B179CD0>, 'train_num_samples': 105, 'test_num_samples': 45, 'num_features': 4}


In [7]:
Weak.generate_M(model_class='pll')
z, w  = Weak.generate_weak(train_y)

In [8]:
print(Weak.__dict__)

{'c': tensor(3, dtype=torch.int32), 'M': array([[0.   , 0.375, 0.375],
       [0.375, 0.   , 0.375],
       [0.375, 0.375, 0.   ],
       [0.25 , 0.25 , 0.25 ]]), 'z': tensor([0, 2, 3, 2, 3, 2, 2, 2, 1, 2, 3, 2, 0, 1, 1, 1, 0, 2, 3, 2, 1, 3, 3, 2,
        0, 2, 1, 3, 2, 1, 1, 2, 2, 0, 1, 0, 3, 2, 0, 2, 2, 0, 1, 0, 2, 1, 0, 3,
        2, 2, 0, 0, 1, 2, 1, 0, 1, 2, 0, 0, 3, 1, 3, 2, 3, 1, 1, 2, 3, 1, 3, 3,
        2, 1, 1, 1, 3, 3, 1, 1, 2, 0, 2, 3, 3, 3, 0, 2, 0, 3, 3, 2, 3, 2, 2, 3,
        1, 2, 1, 0, 3, 0, 1, 3, 3], dtype=torch.int32), 'w': tensor([[0., 1., 1.],
        [1., 1., 0.],
        [1., 1., 1.],
        [1., 1., 0.],
        [1., 1., 1.],
        [1., 1., 0.],
        [1., 1., 0.],
        [1., 1., 0.],
        [1., 0., 1.],
        [1., 1., 0.],
        [1., 1., 1.],
        [1., 1., 0.],
        [0., 1., 1.],
        [1., 0., 1.],
        [1., 0., 1.],
        [1., 0., 1.],
        [0., 1., 1.],
        [1., 1., 0.],
        [1., 1., 1.],
        [1., 1., 0.],
        [1.

In [9]:
#As we are first making a warm up, we will include in the dataset w
DS.include_weak(Weak.w)

In [10]:
trainloader_wu, testloader_wu = DS.get_dataloader()

## Begin the training process

In [11]:
#This is the mlp used by Valen for [mnist,kmnist,fmnist] with random partialization (aka instance independet corruption)
mlp = general_model.MLP(DS.num_features, [DS.num_features, DS.num_features, DS.num_features], DS.num_classes, dropout_p = 0.0, bn = False, seed = 10,
                        layer_init = lambda x: torch.nn.init.kaiming_uniform_(x, a=math.sqrt(5)))

In [12]:
#optim  = torch.optim.SGD(mlp.parameters(), lr = 1e-2, weight_decay = 1e-4, momentum = 0.9)
optim_wu  = torch.optim.SGD(mlp.parameters(), lr = 1e-2, weight_decay = 1e-4, momentum = 0.9)
loss_fn_wu = losses.PartialLoss(Weak.w)

mlp, results_wu = trainig_testing.train_and_evaluate(mlp, trainloader_wu, testloader_wu, optim_wu, loss_fn_wu, num_epochs = 10)



Epoch 1/10: Train Loss: 0.0221, Train Acc: 0.3714, Test Acc: 0.2444
Epoch 2/10: Train Loss: 0.0206, Train Acc: 0.3714, Test Acc: 0.2444
Epoch 3/10: Train Loss: 0.0205, Train Acc: 0.3714, Test Acc: 0.2444
Epoch 4/10: Train Loss: 0.0203, Train Acc: 0.3714, Test Acc: 0.2444
Epoch 5/10: Train Loss: 0.0204, Train Acc: 0.3714, Test Acc: 0.2444
Epoch 6/10: Train Loss: 0.0205, Train Acc: 0.3714, Test Acc: 0.2444
Epoch 7/10: Train Loss: 0.0205, Train Acc: 0.3714, Test Acc: 0.2444
Epoch 8/10: Train Loss: 0.0207, Train Acc: 0.3714, Test Acc: 0.2444
Epoch 9/10: Train Loss: 0.0206, Train Acc: 0.3714, Test Acc: 0.2444
Epoch 10/10: Train Loss: 0.0205, Train Acc: 0.3714, Test Acc: 0.2444


In [13]:
Weak.virtual_labels(train_y)

In [14]:
DS.include_weak(Weak.v)
trainloader, testloader = DS.get_dataloader()

In [15]:
optim  = torch.optim.SGD(mlp.parameters(), lr = 1e-2, weight_decay = 1e-4, momentum = 0.9)
loss_fn = losses.CELoss()

mlp, results = trainig_testing.train_and_evaluate(mlp, trainloader, testloader, optim, loss_fn, num_epochs = 490)

Epoch 1/490: Train Loss: 1.1118, Train Acc: 0.3714, Test Acc: 0.4222
Epoch 2/490: Train Loss: 1.0397, Train Acc: 0.2952, Test Acc: 0.4222
Epoch 3/490: Train Loss: 1.0378, Train Acc: 0.2952, Test Acc: 0.4222
Epoch 4/490: Train Loss: 1.0767, Train Acc: 0.2952, Test Acc: 0.4222
Epoch 5/490: Train Loss: 1.0782, Train Acc: 0.2952, Test Acc: 0.4222
Epoch 6/490: Train Loss: 1.0384, Train Acc: 0.2952, Test Acc: 0.2444
Epoch 7/490: Train Loss: 1.0558, Train Acc: 0.3714, Test Acc: 0.4222
Epoch 8/490: Train Loss: 1.0695, Train Acc: 0.2952, Test Acc: 0.4222
Epoch 9/490: Train Loss: 1.0308, Train Acc: 0.2952, Test Acc: 0.4222
Epoch 10/490: Train Loss: 1.0347, Train Acc: 0.2952, Test Acc: 0.4222
Epoch 11/490: Train Loss: 1.0316, Train Acc: 0.2952, Test Acc: 0.4222
Epoch 12/490: Train Loss: 1.0259, Train Acc: 0.2952, Test Acc: 0.4222
Epoch 13/490: Train Loss: 1.0329, Train Acc: 0.2952, Test Acc: 0.4222
Epoch 14/490: Train Loss: 1.0330, Train Acc: 0.2952, Test Acc: 0.4222
Epoch 15/490: Train Loss: 1.0